In [58]:
source("../src/utils/punto1.R")
source("../src/utils/punto2.R")
source("../src/utils/punto3.R")

In [59]:
tabla_muestra = read.csv("../data/in/muestra.csv")

In [60]:
# Definir parámetros para la prueba
x <- 25  # Edad
r <- 0.05  # Incremento anual del valor asegurado
i <- 0.085  # Tasa de interés
m <- 12  # Pagos mensuales

# Probar la función con la tabla de muestra
print(IrAmx(tabla_muestra, x, r, i, m))
print(ImrAmx(tabla_muestra, x, r, i, m))
print(GrAmx(tabla_muestra, x, r, i, m))

[1] 0.07879583
[1] 0.1022417
[1] 0.1935215
